This project demonstrates a basic PyTorch workflow using the FashionMNIST dataset.
It covers data loading with DataLoader (batch size = 64), model creation using fully connected layers,
training with CrossEntropyLoss and SGD optimizer, and evaluation on test data.
The goal is to provide a clear, beginner-friendly example of building and training a neural network in PyTorch.


In [6]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [7]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:01<00:00, 20.6MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 306kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.66MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 12.0MB/s]


In [8]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"shape of X [N, C, H, W]: {X.shape}")
    print(f"shape of y: {y.shape}")
    break

shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64])


In [10]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
     def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

     def forward(self, x):
      x = self.flatten(x)
      logits = self.linear_relu_stack(x)
      return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [14]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= len(dataloader.dataset)
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [15]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("welldone Patel")

Epoch 1
-------------------------------
loss: 2.307902  [   64/60000]
loss: 2.286206  [ 6464/60000]
loss: 2.266278  [12864/60000]
loss: 2.255149  [19264/60000]
loss: 2.247071  [25664/60000]
loss: 2.228884  [32064/60000]
loss: 2.219669  [38464/60000]
loss: 2.193134  [44864/60000]
loss: 2.183137  [51264/60000]
loss: 2.153675  [57664/60000]
Test Error: 
 Accuracy: 56.5%, Avg loss: 2.150766 

Epoch 2
-------------------------------
loss: 2.165136  [   64/60000]
loss: 2.149737  [ 6464/60000]
loss: 2.093729  [12864/60000]
loss: 2.103497  [19264/60000]
loss: 2.055099  [25664/60000]
loss: 2.013652  [32064/60000]
loss: 2.010032  [38464/60000]
loss: 1.942798  [44864/60000]
loss: 1.936148  [51264/60000]
loss: 1.864876  [57664/60000]
Test Error: 
 Accuracy: 59.3%, Avg loss: 1.868682 

Epoch 3
-------------------------------
loss: 1.907151  [   64/60000]
loss: 1.872995  [ 6464/60000]
loss: 1.756315  [12864/60000]
loss: 1.786600  [19264/60000]
loss: 1.678272  [25664/60000]
loss: 1.653199  [32064/600